In [1]:
%matplotlib notebook
import netCDF4 as nc
import numpy as np
import xie_liu as xl
import matplotlib.pyplot as plt
import pandas as pd
import pickle
path = "dados/maoradflux1long/maoradflux1longM1.c2.20140223.040000.nc"

In [2]:
root = nc.Dataset(path, "r")

In [3]:
    dataset = root
    cf = dataset["cloudfraction_longwave"][:]
    base_time = dataset["base_time"][0]
    time_offset = dataset["time_offset"][:]
    time = (time_offset) / (60 * 60)
    Fdn_clr = dataset["clearsky_downwelling_shortwave"][:]
    Fdn_all = dataset["downwelling_shortwave"][:]
    Fup_all = dataset["upwelling_shortwave"][:]
    Fdn_clr_d = dataset["clearsky_diffuse_downwelling_shortwave"][:]
    Fdn_all_d = dataset["direct_downwelling_shortwave"][:]
    T = dataset["cloud_transmissivity_shortwave"][:]

In [65]:
root

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format UNDEFINED):
    Conventions: ARM-1.1
    command_line: radflux1long_vap -s mao -f M1 -b 20131220 -e 20151201 -n radflux1longc2 require_goodday 0 -D 2
    process_version: vap-radflux1long-2.3-0.el6
    dod_version: radflux1long-c2-1.4
    site_id: mao
    facility_id: M1
    platform_id: radflux1long
    location_description: Green Ocean Amazon (GOAMAZON), Manacapuru, Amazonia, Brazil
    data_level: c2
    rh_factor_flag: 1
    rh_factor_flag_comment: 0 = no_coef_applied 1 = calculate_coef 2 = use_std_coef 3 = use_coef_from_prev_run
    fitmode: 0
    fitmode_comment: 01 = daily_fit 00 =  1_fit
    input_datastreams: maoqcrad1longM1.c2 : 0.5 : 20131220.000000-20151201.000000
    datastream: maoradflux1longM1.c2
    doi: 10.5439/1157585
    history: created by user gaustad on machine amber at 2016-03-18 19:56:26, using vap-radflux1long-2.3-0.el6
    dimensions(sizes): time(1440), bound(2)
    variabl

In [48]:
(B1, B2) = xl.get_B(Fdn_clr, Fdn_all, Fup_all, Fdn_clr_d, Fdn_all_d, T)

/home/danilo/computacoes/ic/xie_liu.py:40: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  B1[B1.mask] = np.nan
/home/danilo/computacoes/ic/xie_liu.py:41: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  B2[B2.mask] = np.nan


In [49]:
result = {"CloudAlbedo":[], "CloudAlbedoCorrected": [], "CloudFraction": [], "CloudFractionCorrected": [], "Time": []}
res = xl.do_work(path)
result["CloudAlbedo"].extend(res[0])
result["CloudAlbedoCorrected"].extend(res[1])
result["CloudFraction"].extend(res[2])
result["CloudFractionCorrected"].extend(res[3])
result["Time"].extend(res[4])
df = pd.DataFrame(result)

/home/danilo/computacoes/ic/xie_liu.py:40: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  B1[B1.mask] = np.nan
/home/danilo/computacoes/ic/xie_liu.py:41: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  B2[B2.mask] = np.nan


In [57]:
alfa = res[0]
alfa_cor = res[1]
inds = (~np.isnan(B1)) & (~np.isnan(alfa))
indsc = (~np.isnan(B1)) & (~np.isnan(alfa_cor))

In [58]:
B1t = B1[inds]
alfat = alfa[inds]
alfact = alfa_cor[inds]
b = B1t / alfat
c = B1t / alfact

In [61]:
np.argmax(b)

178

In [62]:
b[178]

465087856640.00464

In [63]:
B1t[178]

0.84598988

In [64]:
alfat[178]

1.8189894035458383e-12